In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)
os.environ['CUDA_VISIBLE_DEVICES'] = "7"
# os.environ['XLA_FLAGS'] = '--xla_gpu_deterministic_ops=true'

fish-gcp005
/home/akarshkumar0101/nca-alife/src
/home/akarshkumar0101/miniconda3/envs/clip-test/bin/python


In [2]:
from transformers import CLIPProcessor, CLIPModel, FlaxCLIPModel
from PIL import Image
import requests
import jax
from jax import grad, jit
import jax.numpy as jnp
jnp.set_printoptions(linewidth=240)

In [9]:
# pt_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
jax_model = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [10]:
dtype = jnp.float16

In [11]:
jax_model.params = jax_model.to_fp16(jax_model.params);

In [12]:
pixel_values = jnp.zeros((1, 3, 224, 224), dtype=dtype)
inputs = processor(text=["a fat dog"], return_tensors="jax", padding=True)

In [14]:
jax.tree.map(lambda x: x.shape, jax_model.params)

{'logit_scale': (),
 'text_model': {'embeddings': {'position_embedding': {'embedding': (77, 512)},
   'token_embedding': {'embedding': (49408, 512)}},
  'encoder': {'layers': {'0': {'layer_norm1': {'bias': (512,),
      'scale': (512,)},
     'layer_norm2': {'bias': (512,), 'scale': (512,)},
     'mlp': {'fc1': {'bias': (2048,), 'kernel': (512, 2048)},
      'fc2': {'bias': (512,), 'kernel': (2048, 512)}},
     'self_attn': {'k_proj': {'bias': (512,), 'kernel': (512, 512)},
      'out_proj': {'bias': (512,), 'kernel': (512, 512)},
      'q_proj': {'bias': (512,), 'kernel': (512, 512)},
      'v_proj': {'bias': (512,), 'kernel': (512, 512)}}},
    '1': {'layer_norm1': {'bias': (512,), 'scale': (512,)},
     'layer_norm2': {'bias': (512,), 'scale': (512,)},
     'mlp': {'fc1': {'bias': (2048,), 'kernel': (512, 2048)},
      'fc2': {'bias': (512,), 'kernel': (2048, 512)}},
     'self_attn': {'k_proj': {'bias': (512,), 'kernel': (512, 512)},
      'out_proj': {'bias': (512,), 'kernel': (51

In [43]:
jax_model.get_image_features(

dtype('float32')

In [39]:
def loss_fn(pixel_values):
    # inputs = {"pixel_values":pixel_values, "input_ids":input_ids, "attention_mask":attention_mask}
    outputs = jax_model(pixel_values=pixel_values, input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    print(jax.tree.map(lambda x: x.dtype, outputs))
    logits_per_image = outputs.logits_per_image
    return logits_per_image.mean()

grad_fn = jax.grad(loss_fn)

In [40]:
loss_fn(pixel_values).dtype

FlaxCLIPOutput(logits_per_image=dtype('float32'), logits_per_text=dtype('float32'), text_embeds=dtype('float32'), image_embeds=dtype('float32'), text_model_output=FlaxBaseModelOutputWithPooling(last_hidden_state=dtype('float32'), pooler_output=dtype('float32'), hidden_states=None, attentions=None), vision_model_output=FlaxBaseModelOutputWithPooling(last_hidden_state=dtype('float32'), pooler_output=dtype('float32'), hidden_states=None, attentions=None))


dtype('float32')

In [6]:
%timeit loss_fn(pixel_values).block_until_ready()

444 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit grad_fn(pixel_values).block_until_ready()

2024-06-15 20:31:50.859949: E external/xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng41{k2=7,k12=-1,k13=0,k14=4,k15=0,k17=384,k18=1,k22=0,k23=0} for conv (f32[3,1,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[3,768,32,32]{3,2,1,0}, f32[1,768,193,193]{3,2,1,0}), window={size=193x193 pad=192_192x192_192 rhs_reversal=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false} is taking a while...
2024-06-15 20:32:08.223275: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 18.363445219s
Trying algorithm eng41{k2=7,k12=-1,k13=0,k14=4,k15=0,k17=384,k18=1,k22=0,k23=0} for conv (f32[3,1,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[3,768,32,32]{3,2,1,0}, f32[1,768,193,193]{3,2,1,0}), window={size=193x193 pad=192_192x1

6.98 s ± 68.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
loss_fn = jax.jit(loss_fn)
grad_fn = jax.jit(grad_fn)
loss_fn(pixel_values)
grad_fn(pixel_values)

Array([[[[-1.0339806e-02, -1.5528144e-02,  3.0914275e-03, ...,  3.4052439e-03,  1.2733794e-03, -2.3148535e-03],
         [ 1.2935707e-03, -1.0897797e-02,  1.2597716e-02, ...,  1.6456823e-03,  2.0435117e-03, -3.5677245e-04],
         [-1.2817616e-03, -1.5159227e-02,  3.7156881e-03, ...,  9.2984224e-04,  5.3848256e-03, -1.4651179e-03],
         ...,
         [ 2.3531900e-03,  6.8750293e-03,  1.5108207e-03, ..., -4.1287146e-03,  4.2929309e-03,  6.4222170e-03],
         [ 3.1035691e-03,  5.1057828e-03,  9.3761436e-04, ..., -5.4552481e-03,  4.0782429e-03,  6.1401394e-03],
         [-8.1161410e-04, -2.0142768e-03, -5.3532142e-03, ..., -6.5832385e-03,  2.7035226e-03,  3.4725603e-03]],

        [[-3.7530093e-03, -7.2326637e-03,  1.2398622e-02, ...,  2.9629995e-03, -1.4970660e-03, -9.0434691e-03],
         [ 8.3395280e-06, -7.8987908e-03,  1.4534052e-02, ...,  2.3497320e-03,  1.3643214e-03, -5.5640745e-03],
         [-3.8428600e-03, -1.3937127e-02,  3.4129727e-03, ...,  1.1625467e-03,  5.467507

In [9]:
%timeit loss_fn(pixel_values).block_until_ready()

101 ms ± 12.5 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit grad_fn(pixel_values).block_until_ready()

6.11 s ± 237 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
import flax.linen as nn

class FlaxCLIPVisionEmbeddings(nn.Module):
    def setup(self):
        self.patch_embedding = nn.Conv(
            128,
            kernel_size=(3, 3),
            strides=(32, 32),
            padding="VALID",
            use_bias=False,
            # dtype=,
            kernel_init=jax.nn.initializers.normal(),
        )
    def __call__(self, x):
        return self.patch_embedding(x)
net = FlaxCLIPVisionEmbeddings()


In [8]:
rng = jax.random.PRNGKey(0)
x = jnp.zeros((1, 224, 224, 3))
params = net.init(rng, x)

E0615 21:01:46.727392 2427883 buffer_comparator.cc:153] Difference at 1: 7.26737, expected 6.42745
E0615 21:01:46.727420 2427883 buffer_comparator.cc:153] Difference at 2: 6.16159, expected 8.43016
E0615 21:01:46.727423 2427883 buffer_comparator.cc:153] Difference at 5: 8.227, expected 7.17573
E0615 21:01:46.727425 2427883 buffer_comparator.cc:153] Difference at 6: 8.4372, expected 7.24228
E0615 21:01:46.727427 2427883 buffer_comparator.cc:153] Difference at 7: 6.79474, expected 8.67269
E0615 21:01:46.727428 2427883 buffer_comparator.cc:153] Difference at 8: 7.4492, expected 6.16149
E0615 21:01:46.727429 2427883 buffer_comparator.cc:153] Difference at 9: 8.65812, expected 7.46481
E0615 21:01:46.727431 2427883 buffer_comparator.cc:153] Difference at 10: 4.82427, expected 6.95223
E0615 21:01:46.727432 2427883 buffer_comparator.cc:153] Difference at 11: 6.06984, expected 7.39436
E0615 21:01:46.727434 2427883 buffer_comparator.cc:153] Difference at 19: 6.31674, expected 7.49211
2024-06-15 

In [5]:
def loss_fn(x):
    x = net.apply(params, x)
    return x.mean()
grad_fn = jax.grad(loss_fn)

In [6]:
grad_fn(x);